### CR_1.14 - danceable

In [3]:
from pwn import *
from Crypto.Util.Padding import unpad

# Configura il target
HOST = "danceable.challs.cyberchallenge.it"
PORT = 9036

# Funzione per interagire col server e cifrare il messaggio
def encrypt(remote_conn, hex_data):
    remote_conn.sendlineafter("> ", "1")
    remote_conn.sendlineafter("hex)? ", hex_data)
    encrypted = remote_conn.recvline().strip().decode()
    return bytes.fromhex(encrypted)

# 1. Recupera il keystream cifrando 16 byte nulli
conn = remote(HOST, PORT)

# 16 byte nulli
known_plaintext = b"\x00" * 16
known_ciphertext = encrypt(conn, known_plaintext.hex())

# Keystream = ciphertext ⊕ 0x00 = ciphertext
keystream = known_ciphertext[:16]

# 2. Cifra un input vuoto per ottenere il flag cifrato
flag_ciphertext = encrypt(conn, "")  # input vuoto -> plaintext = flag + padding
conn.close()

# 3. Decifra i blocchi del flag
flag_decrypted = b""
for i in range(0, len(flag_ciphertext), 16):
    block = flag_ciphertext[i:i+16]
    # Decifra usando keystream
    decrypted_block = bytes([b ^ k for b, k in zip(block, keystream)])
    flag_decrypted += decrypted_block

# 4. Rimuovi padding PKCS#7
try:
    flag = unpad(flag_decrypted, 16).decode()
    print("Flag:", flag)
except ValueError:
    print("Errore nella rimozione del padding. Dati grezzi:", flag_decrypted)


[x] Opening connection to danceable.challs.cyberchallenge.it on port 9036
[x] Opening connection to danceable.challs.cyberchallenge.it on port 9036: Trying 5.75.221.48
[+] Opening connection to danceable.challs.cyberchallenge.it on port 9036: Done


/var/folders/3n/vf15kv6n675_m6mzk8p8rfxr0000gn/T/ipykernel_35882/128566078.py:10: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  remote_conn.sendlineafter("> ", "1")
/Users/robertovicario/anaconda3/lib/python3.11/site-packages/pwnlib/tubes/tube.py:876: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  res = self.recvuntil(delim, timeout=timeout)
/var/folders/3n/vf15kv6n675_m6mzk8p8rfxr0000gn/T/ipykernel_35882/128566078.py:11: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  remote_conn.sendlineafter("hex)? ", hex_data)


[*] Closed connection to danceable.challs.cyberchallenge.it port 9036
Flag: CCIT{r3us3d_n0nc3_1n_Ch4Ch420}
